#### Import module

In [1]:
import pandas as pd
import os
import glob
import os.path
import numpy as np
from tqdm import tqdm
from sklearn.pipeline import Pipeline
import json
from sklearn.preprocessing import FunctionTransformer
from pathlib import Path


In [3]:
## I/P :Read json files
## O/P : Returns a  json  file as dictionary

def read_json(a_case):
    f = open(a_case)       
    case = json.load(f) 
    return case  
read_json_transformer = FunctionTransformer(read_json)

In [4]:
## I/P :dictionary from each json file
## O/P : Return paragraphs from each json file

def extract_para(case):
    all_para= case['paragraphs']
    paras= []
    para_id =0
    for a_para in all_para:
        para_id = para_id+1
        #### Main Para
        main_para_id = "p_" + str(para_id)
        main_para = a_para["para"][0]          
        paras.append([para_id, main_para_id,main_para]) 
        
        ####  For BlockQuotes
        block_quote =  a_para["blockQuotes"]    
        
        if len(block_quote) != 0:            
            bq = (block_quote[0].split("\n")) 
            bq_count = 0
            for a_bq in bq:
                bq_count = bq_count +1   
                paras.append([para_id, "bq_"+ str(bq_count),a_bq])
                
        #### for Subpara
        sub_para = a_para["subPara"]      
        if len(sub_para) != 0:
            sub_para = list(sub_para.values())
            sub_para_id = 0 
            for a_pre in sub_para:
                sub_para_id =sub_para_id+1
                
                paras.append([para_id, "subpara_"+ str(sub_para_id),a_pre])
                    
    para_df = pd.DataFrame(paras, columns=["ParaID","ContentID","Paragraph"])
        
    return para_df

extract_para_transformer = FunctionTransformer(extract_para)

#### The following pipeline will call both the above function together.

In [5]:
data_pipeline = Pipeline([
    ("read_json", read_json_transformer),
    ("extract_para",extract_para_transformer),    
])

In [6]:
parent_path = Path(r'C:\Users\Nabanita Paul\output\2017\New folder') ## Change the path here . Give the path where json file for each case is available
for a_file in parent_path.rglob('*.json'):    
    para = data_pipeline.fit_transform(a_file)
    path , file_name = os.path.split(a_file)    
    file_name = file_name.replace(".json",".csv")
    a_file = os.path.join(path,file_name,)
    para.to_csv(a_file,index=False)
print("Paragraph Extracted")


Paragraph Extracted


Csv file will be generated for each of the json file given as input 